In [28]:
# Import libraries
import numpy as np
import pandas as pd
import psycopg2
import getpass
import time
import os.path
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
# Load the patient data
_data = np.load('data/patients.npy', allow_pickle=True).tolist()
patients = _data['patients']
print("Loaded the patient data!")

Loaded the patient data!


In [30]:
# Load the database connection details
_data = np.load('data/db_details.npy', allow_pickle=True).tolist()
db_details = _data['db_details']
user = db_details['user']
host = db_details['host']
port = db_details['port']
dbname = db_details['dbname']
schema = db_details['schema']
    
# Connect to the database
conn = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, password=getpass.getpass(prompt='Password:'.format(user)))
cur = conn.cursor()
cur.execute('SET search_path TO {}'.format(schema))
conn.commit()

Password: ··········


In [31]:
# Query the database for radiology reports
query = \
'''
SELECT subject_id, hadm_id, chartdate, description, text
FROM noteevents
WHERE category = 'Radiology'
'''

start = time.time()

# Store the result of the query as a pandas dataframe
reports = pd.read_sql_query(query, conn)

end = time.time()
    
# Print run time
print("The query took {:.1f} seconds".format(end-start))

The query took 15.7 seconds


In [32]:
# Create columns representing the presence of different terms in the radiology reports
reports['bilateral'] = reports['text'].str.lower().str.find('bilateral') >= 0
reports['infiltrates'] = reports['text'].str.lower().str.find('infiltrates') >= 0

# Select reports only for patients we are interested in
reports = reports.loc[reports['hadm_id'].isin(patients['hadm_id'])]

# Add a column for ICU admission time
reports = reports.merge(patients[['hadm_id', 'intime']], on='hadm_id')

# Remove reports taken more than 1 day before or 2 days after ICU admission
diff = reports['chartdate'] - reports['intime']
reports = reports[(diff > np.timedelta64(-1,'D')) & (diff <= np.timedelta64(3,'D'))]

# Show some sample data
reports.head()

,subject_id,hadm_id,chartdate,description,text,bilateral,infiltrates,intime
0,93207,104110.0,2166-10-09,DISTINCT PROCEDURAL SERVICE,"[**Last Name (LF) **],[**First Name3 (LF) **] ...",False,False,2166-10-09 23:07:18
1,93207,104110.0,2166-10-11,CT ABD & PELVIS W/O CONTRAST,[**2166-10-11**] 10:33 AM\n CT ABD & PELVIS W/...,True,False,2166-10-09 23:07:18
2,93207,104110.0,2166-10-09,CTA HEAD W&W/O C & RECONS,[**2166-10-9**] 8:24 PM\n CTA HEAD W&W/O C & R...,True,False,2166-10-09 23:07:18
3,93207,104110.0,2166-10-09,CHEST (PORTABLE AP),[**2166-10-9**] 8:59 PM\n CHEST (PORTABLE AP) ...,False,False,2166-10-09 23:07:18
4,93207,104110.0,2166-10-10,MR HEAD W/O CONTRAST,[**2166-10-10**] 2:39 PM\n MR HEAD W/O CONTRAS...,False,False,2166-10-09 23:07:18


In [33]:
len(np.unique(reports['hadm_id']))

27825

In [34]:
# Select reports that mention both 'bilateral' AND 'infiltrates'
bilateral_infiltrates = reports[reports[['bilateral', 'infiltrates']].all(axis=1)]

# Sort patients by subject_id, then hadm_id, then chartdate
bilateral_infiltrates = bilateral_infiltrates.sort_values(by=['subject_id', 'hadm_id', 'chartdate'])

# Show some sample data
bilateral_infiltrates.head()

,subject_id,hadm_id,chartdate,description,text,bilateral,infiltrates,intime
20758,3,145834.0,2101-10-20,CHEST (PORTABLE AP),[**2101-10-20**] 10:23 PM\n CHEST (PORTABLE AP...,True,True,2101-10-20 19:10:11
52619,96,170324.0,2156-04-02,CHEST (PORTABLE AP),[**2156-4-2**] 5:17 PM\n CHEST (PORTABLE AP) ...,True,True,2156-03-31 16:11:34
52644,96,170324.0,2156-04-02,CHEST (PORTABLE AP),[**2156-4-2**] 5:13 AM\n CHEST (PORTABLE AP) ...,True,True,2156-03-31 16:11:34
48505,209,135807.0,2119-09-25,CHEST (PORTABLE AP),[**2119-9-25**] 2:57 PM\n CHEST (PORTABLE AP) ...,True,True,2119-09-25 09:34:10
38218,214,197273.0,2188-10-07,CHEST (PORTABLE AP),[**2188-10-7**] 10:34 PM\n CHEST (PORTABLE AP)...,True,True,2188-10-06 23:56:53


In [36]:
len(np.unique(bilateral_infiltrates['hadm_id']))

1339

In [18]:
# Print a sample radiology report
print(bilateral_infiltrates.loc[48505,'text'])

[**2119-9-25**] 2:57 PM
 CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 107240**]
 Reason: s/p CABG
 Admitting Diagnosis: UNSTABLE ANGINA
 ______________________________________________________________________________
 [**Hospital 4**] MEDICAL CONDITION:
   65 year old man s/p CABG
 REASON FOR THIS EXAMINATION:
  s/p CABG
 ______________________________________________________________________________
                                 FINAL REPORT
 CHEST AP PORTABLE SINGLE VIEW:

 INDICATION:  Status post bypass surgery.

 FINDINGS:  AP single view of the chest obtained with patient in semi-erect
 position demonstrates ETT in place terminating in the trachea some 6 cm above
 the carina.  An NG tube has been passed and reaches safely below the diaphragm
 terminating in the area of the fundus.  A right jugular approach central
 venous line carries a Swan-Ganz catheter, tip of which reaches the main PA.
 One mediastinal tube and a left side

In [20]:
# Show reports for confirmed ARDS
ards_hadm_id = patients[patients['diagnosis'].str.lower().str.find('ards') >= 0]['hadm_id']
confirmed_ards = bilateral_infiltrates.loc[bilateral_infiltrates['hadm_id'].isin(ards_hadm_id)]
confirmed_ards.head()

,subject_id,hadm_id,chartdate,description,text,bilateral,infiltrates,intime
18305,818,154890.0,2157-08-10,CHEST (PORTABLE AP),[**2157-8-10**] 6:45 PM\n CHEST (PORTABLE AP) ...,True,True,2157-08-10 18:03:00
41831,6588,171653.0,2136-01-19,CHEST (PORTABLE AP),[**2136-1-19**] 5:56 AM\n CHEST (PORTABLE AP) ...,True,True,2136-01-16 20:04:04
55103,8346,173640.0,2114-03-21,CHEST (PORTABLE AP),[**2114-3-21**] 5:44 AM\n CHEST (PORTABLE AP) ...,True,True,2114-03-21 06:16:15
55101,8346,173640.0,2114-03-23,CHEST (PORTABLE AP),[**2114-3-23**] 8:06 AM\n CHEST (PORTABLE AP) ...,True,True,2114-03-21 06:16:15
4352,8879,107833.0,2164-07-28,CHEST (PORTABLE AP),[**2164-7-28**] 2:09 PM\n CHEST (PORTABLE AP) ...,True,True,2164-07-28 13:41:00


In [22]:
print(confirmed_ards.loc[18305,'text'])

[**2157-8-10**] 6:45 PM
 CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 36456**]
 Reason: transfer from [**Location (un) **] to [**Hospital Ward Name **] MICU with hx of A
 ______________________________________________________________________________
 [**Hospital 2**] MEDICAL CONDITION:
  36 year old woman with ARDS
 REASON FOR THIS EXAMINATION:
  transfer from [**Location (un) **] to [**Hospital Ward Name **] MICU with hx of ARDS
  please check tubes, lines, and eval bilateral infiltrates
 ______________________________________________________________________________
                                 FINAL REPORT
 PORTABLE AP SUPINE CHEST RADIOGRAPH

 INDICATION:   Evaluate tubes, lines.

 No comparison is available.

 There are bilateral diffuse opacities. The cardiac silhouette is at the upper
 limits of normal. The pulmonary vascularity is not well evaluated.  There is
 [**Hospital1 **]-apical pleural thickening, possibly represent

In [21]:
len(confirmed_ards)

23